In [659]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue

In [660]:
# constants
PROBLEM_SIZE = 5  # dimension of the finite set U
NUMBER_SET = 10  # number of subsets in the collection S
SETS = tuple(
    np.array([random() < 0.3 for i in range(PROBLEM_SIZE)]) for j in range(NUMBER_SET)
)  # generate sets in S

# Define State as a named tuple
State = namedtuple("State", ["taken", "cost", "heuristic"])


In [661]:
def goal_check(state, sets):
    """
    check if the logical OR all the elements yeald a line of all true ie the
    condition for a state to be covering the whole set U
    """
    return np.all(
        reduce(np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE))
    )


# assert generated problem is solvable, ie the goal check of a stete with all
# sets taken is true
assert goal_check(State(range(NUMBER_SET), 0, 0), SETS)

In [662]:
def cost(state):
    """The cost function calculates the cost of reaching a particular state"""
    return len(state.taken)

Commonly used Heuristic for set covering:

- Minimum Remaining Elements (MRE): Prioritizes subsets with the fewest uncovered elements.

- Maximum Coverage (MC): Favors subsets that cover the maximum number of uncovered elements.

- Combination of MRE and MC: Balances between subsets with few remaining elements and high coverage.

- Greedy Approximation (GA): Minimizes the cost-to-coverage ratio when selecting subsets.

- Randomized Heuristics: Selects subsets randomly or based on probabilistic considerations to explore different paths.

In [663]:
def MRE_heuristic(state, sets):
    """
    - Objective: The MRE heuristic prioritizes selecting subsets that have the
      fewest remaining uncovered elements.
    - Selection Criterion: It selects subsets that cover elements that are closest
    to being fully covered. In other words, it aims to quickly cover the remaining
    elements.
    - Impact: The primary goal is to minimize the number of remaining uncovered
    elements, which can lead to a different subset selection strategy compared to
    the MC heuristic.
    """
    uncovered = reduce(
        np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE)
    )
    # calculates number of uncovered elements and assign the ones wiht less
    # higher priority
    priority = -np.sum(np.logical_not(uncovered))
    return priority


def MC_heuristic(state, sets):
    """
    - Objective: The MC heuristic prioritizes selecting subsets that cover the
    maximum number of currently uncovered elements.
    - Selection Criterion: It selects subsets that contribute to covering the largest
    number of currently uncovered elements.
    - Impact: The primary goal is to maximize
    the coverage with each selection, potentially reducing the total number of
    subsets needed to cover all elements.
    """
    uncovered = reduce(
        np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE)
    )
    # calculates number of uncovered elements and assign the ones wiht less
    # higher priority
    priority = -np.sum(uncovered)
    return priority


def MRE_MC_heuristic(state, sets):
    """
    Prioritize with a balanced approach between MRE and MC
    """
    return (MRE_heuristic(state, sets) + MC_heuristic(state, sets)) / 2


def GA_heuristic(state, sets):
    """
    Prioritize with cost/coverage ration
    """
    uncovered = reduce(
        np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE)
    )

    # TODO
    # calculate the cost/coverage ratio



def RANDOM_heuristic(state, sets):
    """
    Prioritize with a random heuristic
    """
    return random()


In [664]:
def astar(sets, heuristic):
    # Initialize the priority queue with the initial state
    initial_state = State(
        taken=[],
        cost=0,
        heuristic=heuristic(State(taken=[], cost=0, heuristic=0), sets),
    )
    open_set = PriorityQueue()
    open_set.put((initial_state.cost + initial_state.heuristic, initial_state))

    # Initialize the closed set as an empty set
    closed_set = set()

    checked_states=0

    while not open_set.empty():
        # Get the state with the lowest f score from the priority queue
        _, current_state = open_set.get()

        checked_states+=1

        # If the current state is a goal state, return the solution
        if goal_check(current_state, sets):
            return current_state.taken,checked_states

        # Add the current state to the closed set
        closed_set.add(tuple(current_state.taken))

        # Generate successor states by adding one more subset
        for subset in range(NUMBER_SET):
            if subset not in current_state.taken:
                # Create a new state by adding the subset
                new_taken = current_state.taken + [subset]
                new_cost = cost(State(new_taken, 0, 0))
                new_heuristic = heuristic(State(new_taken, 0, 0), sets)
                new_state = State(new_taken, new_cost, new_heuristic)

                # If the state is not in the closed set, add it to the open set
                if tuple(new_taken) not in closed_set:
                    open_set.put((new_state.cost + new_state.heuristic, new_state))

    # If the open set is empty and no solution is found, return None
    return None


In [665]:
for i in range(NUMBER_SET):
    print("Set",i,SETS[i],"Coverage:",np.sum(SETS[i]))

print()
print("MRE:",astar(SETS, MRE_heuristic))
print("MC:",astar(SETS, MC_heuristic))
print("MRE+MC:",astar(SETS, MRE_MC_heuristic))
print("RANDOM:",astar(SETS, RANDOM_heuristic))
print("GA:",astar(SETS, GA_heuristic))

Set 0 [False  True False False False] Coverage: 1
Set 1 [False False False False False] Coverage: 0
Set 2 [False False False False  True] Coverage: 1
Set 3 [False False False  True  True] Coverage: 2
Set 4 [False False False False False] Coverage: 0
Set 5 [ True  True  True False False] Coverage: 3
Set 6 [False False  True False  True] Coverage: 2
Set 7 [False False False  True False] Coverage: 1
Set 8 [False  True False False  True] Coverage: 2
Set 9 [False False False False  True] Coverage: 1

MRE: ([3, 5], 2849)
MC: ([5, 3], 3)
MRE+MC: ([3, 5], 43)
RANDOM: ([3, 5], 26)


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'